In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 75.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 5.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 99.0 MB

In [3]:
from vllm import LLM, SamplingParams
# 提示词列表
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "Once Upon the time",
    "If you have a prety baby",
]

# 更详细的采样参数配置
sampling_params = SamplingParams(
    temperature=0.8,          # 随机性控制
    top_p=0.95,               # 核采样阈值
    max_tokens=100,           # 最大生成token数
    stop=["\n", ".", "?"],    # 停止符
    repetition_penalty=1.1    # 重复惩罚，减少重复生成
)

# 初始化LLM，添加优化参数
llm = LLM(
    model="facebook/opt-1.3b",
    tensor_parallel_size=1,   # 张量并行数量，根据GPU数量调整
    gpu_memory_utilization=0.8, # 允许使用的GPU内存比例
    max_num_batched_tokens=2048, # 批处理最大token数
)

# 进行推理
outputs = llm.generate(prompts, sampling_params)

# 打印格式化输出
for i, output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"示例 {i+1}:")
    print(f"提示: {prompt!r}")
    print(f"生成: {generated_text!r}\n")

2025-07-24 06:49:23.854880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753339764.184976      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753339764.281204      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 07-24 06:49:41 [__init__.py:244] Automatically detected platform cuda.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

INFO 07-24 06:49:58 [config.py:841] This model supports multiple tasks: {'generate', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 07-24 06:49:58 [config.py:1472] Using max model len 2048
WARNING 07-24 06:49:58 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 07-24 06:49:59 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='facebook/opt-1.3b', speculative_config=None, tokenizer='facebook/opt-1.3b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disa

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-24 06:51:18 [model_runner.py:1671] Graph capturing finished in 35 secs, took 0.18 GiB
INFO 07-24 06:51:18 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 42.50 seconds


Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

示例 1:
提示: 'Hello, my name is'
生成: ' Joel'

示例 2:
提示: 'The president of the United States is'
生成: ' a pretty big deal'

示例 3:
提示: 'The capital of France is'
生成: ' Paris'

示例 4:
提示: 'The future of AI is'
生成: ' now'

示例 5:
提示: 'Once Upon the time'
生成: ', there was a fun little place in Memphis called the Blues on Broadway'

示例 6:
提示: 'If you have a prety baby'
生成: ' and you’re still struggling to get in shape, there are some simple steps that can help'



In [2]:
# 查看 CUDA 工具包版本（nvcc 编译器版本）
!nvcc --version
# 查看 PyTorch 对应的 CUDA 版本
import torch
print("PyTorch 使用的 CUDA 版本:", torch.version.cuda)
#  确保系统 CUDA 版本 ≥ PyTorch 依赖版本 ， 否则需要升级系统CUDA版本或降级PyTorch依赖版本

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
PyTorch 使用的 CUDA 版本: 12.6
